## Imports

In [2]:
import azureml.core
# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config(path='azure_config_dev.json')
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')



Azure ML SDK Version:  1.1.5
twv_test-ws	australiaeast	POCML	australiaeast


In [3]:
ml_service = 'PAD'

#TODO - replace with call to HRXML Config API to get algorithm details for web service
if ml_service == 'TWV':
    algorithm = 'DNN_MLPRegressor'
    description = 'Tax Withholding Verification'
elif ml_service == 'PAD':
    algorithm = 'Isolation_Forest'
    description = 'Payroll Anomaly detection'
else:
    algorithm = 'Other algorithm'
    description = 'Other'

###  Choose published models to deploy in web service, and try  to download to ensure they exist

In [31]:
models_list = [{'ml_service':'TWV','model_code':'SS01','model_version':'010','vers':1},{'ml_service':'TWV','model_code':'T001','model_version':'002','vers':1},{'ml_service':'PAD','model_code':'M010','model_version':'002','vers':1},{'ml_service':'PAD','model_code':'M005','model_version':'001','vers':1},{'ml_service':'PAD','model_code':'A001','model_version':'001','vers':5}] #,{'model_code':'T003','model_vers':2}]
#ml_service = 'TWV'


twv_models = []

for model_details in models_list:
    ml_serv = model_details['ml_service']
    model = model_details['model_code']  #'T003'
    model_version = None
    if 'model_version' in model_details:
        model_version = model_details['model_version']
    version = model_details['vers']
    if model_version is None:
        twv_model=Model(ws, ml_serv + '_model_' + model,version=version)
    else:
        twv_model=Model(ws, ml_serv + '_model_' + model + '_' + model_version,version=version)
    twv_model.add_tags({'azure_model_vers':version})
    twv_models.append(twv_model)

    print('model: ' + str(twv_model.name) + ' ' + str(twv_model.version))

    ser = twv_model.serialize()
    print('ser: ' + str(ser))
    twv_model.download(target_dir=os.path.join(os.getcwd(),'downloaded_models'), exist_ok=True)

    # verify the downloaded model file
    #file_path = os.path.join(os.getcwd(), "sklearn_mnist_model.pkl")

    #os.stat(file_path)

Replacing tag azure_model_vers -> 1 with azure_model_vers -> 1
Model tag add operation complete.
model: TWV_model_SS01_010 1
ser: {'createdTime': '2019-11-01T05:19:48.928468+00:00', 'createdBy': None, 'description': None, 'id': 'TWV_model_SS01_010:1', 'mimeType': 'application/json', 'name': 'TWV_model_SS01_010', 'framework': 'Custom', 'frameworkVersion': None, 'tags': {'RunNumber': '1', 'RunId': 'train_TWV_SS01_010_1572585323_098301fe', 'Model': 'SS01', 'ModelVersion': '010', 'HiddenLayers': '2', 'LearningRate': '0.001', 'TrainSetSize': '2374', 'Algorithm': 'DNN_MLPRegressor', 'Iters': '2000', 'NumFeatures': '20', 'azure_model_vers': 1}, 'properties': {}, 'unpack': False, 'url': 'aml://asset/b10c9aff90e64667a2c9b6d31b5cf9af', 'version': 1, 'experimentName': None, 'runId': None, 'runDetails': 'None', 'datasets': {}, 'resourceConfiguration': None, 'sampleInputDatasetId': None, 'sampleOutputDatasetId': None}
Replacing tag azure_model_vers -> 1 with azure_model_vers -> 1
Model tag add oper

## Package dependencies

In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("pandas")
myenv.add_conda_package("matplotlib")
myenv.add_conda_package("numpy")
myenv.add_conda_package("requests")
myenv.add_channel("conda-forge")
myenv.add_pip_package("azureml-pipeline-core")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())


In [6]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

  - azureml-pipeline-core
- scikit-learn
- pandas
- matplotlib
- numpy
- requests
channels:
- anaconda
- conda-forge



## Web service depoyment configuration

In [7]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"ml_service": ml_service,"algorithm":algorithm}, 
                                               description=description)

# Option 1 - Use when updating an  existing web service with a new image

In [8]:
ngamlfpy_package = 'ngamlfpy'

In [9]:
ml_service = 'TWV' # for image purposes - has both pad and twv modesl in twvimage

In [10]:
%%time
#rbm code to create new image

from azureml.core.image import Image
from azureml.core.image import ContainerImage

new_img_name = ml_service.lower() + 'image'
image_config = ContainerImage.image_configuration(execution_script="predict_model.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml",
                                                 dependencies=[os.path.join('.',ngamlfpy_package)
                                                               #os.path.join('.', 'gen_utils.py'),
                                                               #os.path.join('.', 'pipeline_utils.py'),
                                                               #os.path.join('.',  'train_utils.py')
                                                               ])


img = Image.create(workspace=ws, name=new_img_name, models=twv_models, image_config=image_config)

C:\Users\russellM\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  # Remove the CWD from sys.path while we load stuff.
C:\Users\russellM\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments


Creating image
Wall time: 9.1 s


In [18]:
#rbm code to get webservice for existing image
from azureml.core.webservice import Webservice

#prev_image_name = 'twvdnn9' 
web_services = Webservice.list(workspace=ws, image_name=None, image_id=None, model_name=None, model_id=None, tags=None, properties=None)
web_services


[AciWebservice(workspace=Workspace.create(name='twv_test-ws', subscription_id='0c7f1ae6-7228-403f-ba54-84e72c46d6cf', resource_group='POCML'), name=twvimage, image_id=twvimage:23, compute_type=sha256:3149a900ad933a27ca72cc52a058087d450812a4d4606bebb586ceab7ad1360b, state=ACI, scoring_uri=None, tags=http://b03d85f2-b070-497e-a0f8-03ec2315c0af.westus.azurecontainer.io/score, properties={'ml_service': 'TWV', 'model_code': 'T001', 'algorithm': 'DNN_MLPRegressor'}, created_by={'azureml.git.repository_uri': 'https://github.com/russellmyers/nga_mlpy.git', 'mlflow.source.git.repoURL': 'https://github.com/russellmyers/nga_mlpy.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '94bee109d41c2ec247d8840f44551f157ec2aa0b', 'mlflow.source.git.commit': '94bee109d41c2ec247d8840f44551f157ec2aa0b', 'azureml.git.dirty': 'True'}),
 AciWebservice(workspace=Workspace.create(name='twv_test-ws', subscription_id='0c7f1ae6-7228-403f-ba54-84e72c46d6cf', resource_gr

In [20]:
web_services[0].name

'twvimage'

In [21]:
images=Image.list(workspace=ws)
images

C:\Users\russellM\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  """Entry point for launching an IPython kernel.


[ContainerImage(workspace=Workspace.create(name='twv_test-ws', subscription_id='0c7f1ae6-7228-403f-ba54-84e72c46d6cf', resource_group='POCML'), name=twvimage, id=twvimage:23, tags={}, properties={'azureml.git.repository_uri': 'https://github.com/russellmyers/nga_mlpy.git', 'mlflow.source.git.repoURL': 'https://github.com/russellmyers/nga_mlpy.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '94bee109d41c2ec247d8840f44551f157ec2aa0b', 'mlflow.source.git.commit': '94bee109d41c2ec247d8840f44551f157ec2aa0b', 'azureml.git.dirty': 'True'}, version=23),
 ContainerImage(workspace=Workspace.create(name='twv_test-ws', subscription_id='0c7f1ae6-7228-403f-ba54-84e72c46d6cf', resource_group='POCML'), name=sdsimage, id=sdsimage:1, tags={}, properties={}, version=1),
 ContainerImage(workspace=Workspace.create(name='twv_test-ws', subscription_id='0c7f1ae6-7228-403f-ba54-84e72c46d6cf', resource_group='POCML'), name=twvimage, id=twvimage:22, tags={}, prop

In [22]:
new_img = images[0]
new_img

ContainerImage(workspace=Workspace.create(name='twv_test-ws', subscription_id='0c7f1ae6-7228-403f-ba54-84e72c46d6cf', resource_group='POCML'), name=twvimage, id=twvimage:23, tags={}, properties={'azureml.git.repository_uri': 'https://github.com/russellmyers/nga_mlpy.git', 'mlflow.source.git.repoURL': 'https://github.com/russellmyers/nga_mlpy.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '94bee109d41c2ec247d8840f44551f157ec2aa0b', 'mlflow.source.git.commit': '94bee109d41c2ec247d8840f44551f157ec2aa0b', 'azureml.git.dirty': 'True'}, version=23)

In [23]:
#rbm code to update existing webservie with new image
web_services[0].update(image=new_img, tags=None, properties=None, description=ml_service + " Prediction", auth_enabled=None, ssl_enabled=None, ssl_cert_pem_file=None, ssl_key_pem_file=None, ssl_cname=None, enable_app_insights=None)

In [24]:
web_services[0].get_logs()

"2020-03-30T03:41:26,552540184+00:00 - gunicorn/run \n2020-03-30T03:41:26,552515483+00:00 - rsyslog/run \n2020-03-30T03:41:26,553684825+00:00 - iot-server/run \n2020-03-30T03:41:26,562810556+00:00 - nginx/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2020-03-30T03:41:26,847358776+00:00 - iot-server/finish 1 0\n2020-03-30T03:41:26,856885222+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.9.0\nListening at: http://127.0.0.1:31311 (9)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 44\ngenerated new fontManager\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user's init function\nUsers's init has completed successfully\n/opt/miniconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this func

# ... or Option 2) Use when deplying a new web service (also deploys new image at same time)

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

ngamlfpy_package = 'ngamlfpy'

# configure the image
image_config = ContainerImage.image_configuration(execution_script="predict_model.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml",
                                                 dependencies=[
                                                               os.path.join('.',ngamlfpy_package)
                                                               #os.path.join('.', 'gen_utils.py'),
                                                               #os.path.join('.', 'pipeline_utils.py'),
                                                               #os.path.join('.',  'train_utils.py')
                                                               ])

service = Webservice.deploy_from_model(workspace=ws,
                                       name= ml_service.lower() + 'image',
                                       deployment_config=aciconfig,
                                       models=twv_models,
                                       image_config=image_config)


service.wait_for_deployment(show_output=True)

## Note: can debug prediction web service with service.get_logs()

In [ ]:
print(service.get_logs())

## Try running web service

In [26]:
web_services[0].scoring_uri

'http://b03d85f2-b070-497e-a0f8-03ec2315c0af.westus.azurecontainer.io/score'

In [ ]:
import json
model = 'T001'
predict_file_name =  model + '_test.json'
data_dir='data/predict'
predict_source = 'json'
model_prefix = model
predict_file_path = '/'.join([data_dir,model_prefix])
full_predict_file_name = '/'.join([predict_file_path,predict_file_name])
print ('Predict file: ',full_predict_file_name)
if predict_source == 'json':
    with open(full_predict_file_name) as json_data:
        j_data = json.load(json_data)
raw_data = {}
raw_data['data'] = j_data
raw_data['data']

In [ ]:
raw_data_str = json.dumps(raw_data)
test_samples = bytes(raw_data_str, encoding='utf8')
result = web_services[0].run(input_data=test_samples)

result

In [ ]:
# import json


# predict_file_name = model + '_test.json'
# data_dir='data/predict'
# predict_source = 'json'
# model_prefix = model + '_' + target_vers
# predict_file_path = '/'.join([data_dir,model_prefix])
# full_predict_file_name = '/'.join([predict_file_path,predict_file_name])
# print ('Predict file: ',full_predict_file_name)
# if predict_source == 'json':
#     with open(full_predict_file_name) as json_data:
#         j_data = json.load(json_data)
        

# test_samples = json.dumps({"data": j_data})
# test_samples = bytes(test_samples, encoding='utf8')

# # predict using the deployed model
# result = service.run(input_data=test_samples)
# result